# Manual Brand Report Refresh - DCM

In [10]:
!git checkout dev
from brand_reporting_dags.dfa_reporting_dag_tasks import dfa_report_extract,dfa_report_load,clean_up

import json
import os
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")

M	Brand Reporting Refresh.ipynb
M	bonito_Q3_refresh.ipynb
D	dag playground.ipynb
M	rtf_utils/__pycache__/dfa_utils.cpython-36.pyc
M	rtf_utils/__pycache__/gcp_utils.cpython-36.pyc
Already on 'dev'
Your branch is up to date with 'origin/dev'.


In [8]:
from rtf_utils.dfa_utils import CampaignManagerReport,get_dfa_report,clean_dcm_file
from rtf_utils.gcp_utils import BigQuery,CloudStorage
from essence.analytics.platform import securedcredentials as secure_creds

## DCM Reports

In [25]:
with open('dags/brand_report_task_queue.json') as json_file:
    config_json = json.load(json_file)
    campaigns = config_json['campaigns']

Task = (report_id, bq_dest (proj:dataset.table), schema)

In [3]:
tasks = campaigns[0]['dcmTasks']

In [5]:
start_date = '2019-09-01'
end_date = '2019-09-10'
gcs_files = [dfa_report_extract(task['reportId'],start_date = start_date,end_date = end_date) for task in tasks]

Get Report Meta
File is already available. Start download
File downloaded to Bonito_Classification.csv
Cleaning File
Auth GCS
Upload File
Clean Up Local
Get Report Meta
File doesn't exist, run report
The Download Part
Report Processing, Wait 10. Try 0
Report Processing, Wait 10. Try 1
Print Report Available!
File downloaded to Bonito_Video_Player_Size.csv
Cleaning File
Auth GCS
Upload File
Clean Up Local


In [4]:
gcs_files = [
('rtf_staging', 'brand_reporting/2019-09-10_Bonito_Classification.csv'),
('rtf_staging', 'brand_reporting/2019-09-10_Bonito_Video_Player_Size.csv')]

### DCM Report Loader
#### Note: Reports need to be cleaned

In [11]:
uris = ['gs://rtf_staging/5295_BR_Reporting_Classification_20190913_142345_2656806431.csv',
        'gs://rtf_staging/5295_BR_Reporting_Geo_Report_20190913_144251_2656783687.csv',
        'gs://rtf_staging/5295_BR_Reporting_Video_Player_Size_20190913_140102_2656781516.csv'
       ]

service_account_email='131786951123-compute@developer.gserviceaccount.com'
credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)
bq = BigQuery(credentialsFromVault)

for file in uris:#gcs_files:
    
    #bucket, filename = file
    #file_uri = "gs://" + bucket + "/" + filename
    
    file_uri = file
    
    dataset_id = 'RTF_DWH_CampaignManager'
    
    if 'Classification' in file_uri:
        dest_table = 'DCM_contentReport'
    elif 'Geo' in file_uri:
        dest_table = 'DCM_geoReport' 
    else:
        dest_table = 'DCM_playerSizeReport'
    print("Loading {} into {}".format(file_uri,dest_table))
    bq.load_from_gcs(dataset_id,file_uri,dest_table,mode='Append')

Loading gs://rtf_staging/5295_BR_Reporting_Classification_20190913_142345_2656806431.csv into DCM_contentReport
Loading gs://rtf_staging/5295_BR_Reporting_Geo_Report_20190913_144251_2656783687.csv into DCM_geoReport
Loading gs://rtf_staging/5295_BR_Reporting_Video_Player_Size_20190913_140102_2656781516.csv into DCM_playerSizeReport


In [61]:
def task_orchestrator(campaign_list,):
    dcm_tasks = []
    
    for campaign in config_json['campaigns']:
         dcm_tasks.extend(campaign['dcmTasks'])


    prev_task_name = ""

    tasks_sorted = sorted(dcm_tasks, key = lambda i: i['name'])

    while tasks_sorted:
        task = tasks_sorted.pop()
        task_name = task['name']
        if task_name != prev_task_name:
            print("Starting {}".format(task_name))
        prev_task_name = task_name
        print(task['reportId'])


Starting video_player_size



632709822
Starting verification_report



631165400
Starting geo_report



633170439


In [1]:
def clean_dcm_file(filename):
    data = []
    write = False
    with open(filename,'r') as f:
        reader = csv.reader(f, delimiter=',')

        for row in reader:
            if write == True:
                data.append(row)        
            elif row == ['Report Fields']:
                write = True

        if data[-1][0] == 'Grand Total:':
            data.pop()

    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(data)

In [6]:
import csv
clean_dcm_file('/Users/kylerandolph/Downloads/5295_BR_Reporting_Video_Player_Size_20190913_140102_2656781516.csv')

In [16]:
import datetime
datetime.datetime.now().strftime('%Y%m%d')

'20190913'

In [22]:
r = ("RTF_DWH_CampaignReporting." + 
     "task" + 
     datetime.datetime.now().strftime('%Y%m%d'))

In [23]:
r

'RTF_DWH_CampaignReporting.task20190913'

In [24]:
"hei {}".format(1)

'hei 1'